# Advent of Code 2023

## Day 7 -- Camel Cards (Part 2)

## Author: Chris Kimber

The instructions for this problem can be found at https://adventofcode.com/2023/day/7.

Thanks to u/LxsterGames on the AoC subreddit for posting a sample input that tested a number of edge cases the provided sample input did not. Helped me debug something!

The loading and parsing of the data is the same as it was in part 1.

In [248]:
with open("input", "r") as f:
    input_list = f.read().splitlines()

In [249]:
input_list = [x.split(" ") for x in input_list]

In [250]:
for i in input_list:
    i[1] = int(i[1])

The ranking of individual cards is changed slightly in part 2, so that J is now the lowest-ranked card.

In [231]:
c_rank = {"A": 12, "K": 11, "Q": 10, "J": 0, "T": 9, "9": 8, "8": 7, "7": 6, "6": 5, "5": 4, "4": 3, "3": 2, "2": 1}

There are three main cases that occur when J is wild. The jack_checker() function checks which of these 3 cases a hand matches and proceeds accordingly.

The first case is if there are no J in a hand: in this case, a similar hand ranking function to part 1 is used with minor adjustments to reflect the fact that it now is meant to run within wrapper functions that handle the wildcard nature of J. This is hand_scorer(). The same function is also used after the transformations performed by the following two functions to actually rank the hands.

The second case is if J is the most common card: in this case, the second most common card's count is added to J. This is always the move that gives the highest hand ranking when J is the most common card. If the hand is already 5 J, this can be skipped (and it gets the max ranking). This is what jack_max() does.

The third case is if there are one or more Js in the hand but J is not the most common card: In this case, the count of J is added to that of the most common card. This is always the move that gives the highest hand ranking in this case. This is what jack_elsewhere() does.

In [232]:
from collections import Counter

In [233]:
def hand_scorer(c_mc):
    if c_mc[0][1] == 5:
        hrank = 6 # five of a kind
    elif c_mc[0][1] == 4:
        hrank = 5 # four of a kind
    elif c_mc[0][1] == 3:
        if c_mc[1][1] == 2:
            hrank = 4 # full house
        else:
            hrank = 3 # three of a kind
    elif c_mc[0][1] == 2:
        if c_mc[1][1] == 2:
            hrank = 2 # two pair
        else:
            hrank = 1 # one pair
    else:
        hrank = 0 # high card
    return hrank

In [234]:
def jack_max(c, c_mc): #jack is max1
    if c_mc[0][1] != 5:
        max2 = c_mc[1][0]
        c["J"] = c["J"] + c[max2]
        c[max2] = 0
        c_mc = c.most_common()
    return c_mc

In [235]:
def jack_elsewhere(c, c_mc):
    max1 = c_mc[0][0]
    c[max1] = c[max1] + c["J"]
    c["J"] = 0
    c_mc = c.most_common()
    return c_mc

In [238]:
def jack_checker(hand):
    c = Counter(hand)
    c_mc = c.most_common()
    if c["J"] == 0:
        # run original scorer (hand_scorer)
        hrank = hand_scorer(c_mc)
    elif c_mc[0][0] == "J":
        # add max2 card to jack (jack_max)
        # then run hand_scorer
        c_mc = jack_max(c, c_mc)
        hrank = hand_scorer(c_mc)
    else:
        # add jack to max1 card (jack_elsewhere)
        # then run hand_scorer
        c_mc = jack_elsewhere(c, c_mc)
        hrank = hand_scorer(c_mc)
    return hrank

The rest of the code is effectively identical to part 1. The jack_checker() wrapper is applied to each hand, the hands are ranked by hand type and then each card is ranked in order. The hands are multiply sorted based on these rankings such that the overall ranking is given by the index+1, then the winnings per hand are calculated and summed for total winnings; this is the answer to the problem!

In [251]:
for i in input_list:
    hrank = jack_checker(i[0])
    i.append(hrank)

In [240]:
def card_comparer(hand):
    for i in hand[0]:
        hand.append(c_rank[i])
    return hand

In [252]:
for i in input_list:
    card_comparer(i)

In [242]:
from operator import itemgetter

In [253]:
sorted_list = sorted(input_list, key = itemgetter(2,3,4,5,6,7))

In [254]:
winnings = []
for i,hand in enumerate(sorted_list):
    rank = i+1
    bid = hand[1]
    winnings.append(rank*bid)

In [255]:
total_winnings = sum(winnings)

In [256]:
print(total_winnings)

249138943
